In [7]:
def mac_matrix(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    print(mons)
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    return matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs])


def recover_polys(M, d, ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B) # Sage doesn't know how to work with the output list unless it is a "Sequence" of polynomials


def mac_basis(f,d,ring):
    mons0 = monomials(list(ring.gens()), [d+1 for i in range(len(ring.gens()))])
    mons = []
    for mon in mons0:
        if mon.degree() <= d:
            mons.append(mon)
    mons.sort()
    mons.reverse() # for reasons I don't understand, the .sort() method respects the chosen order but reverses it
    col_labs = []
    for deg in range(d + 1):
        segment = []
        for poly in f:
            for mon in mons:
                label = ring(mon)*ring(poly)
                if label.degree() == deg:
                    segment.append(label)
        col_labs += segment
    M = matrix([[label.monomial_coefficient(mon) for mon in mons] for label in col_labs]).rref()
    B = []
    for row in M.rows():
        g = 0
        for (coeff,mon) in zip(row,mons):
            g += coeff*mon
        if g != 0:
            B.append(g)
    if B == []:
        B = [ring(0)]
    return Sequence(B)


def mac_grob(f,ring):
    test_deg = 0
    while True:
        B = mac_basis(f, test_deg, ring)
        print(test_deg, B, B.is_groebner(), ideal(B) == ideal(f))
        if B.is_groebner() and ideal(B) == ideal(f):
            return (test_deg,B)
        test_deg += 1

In [8]:
R.<x,y> = PolynomialRing(QQ, order = "degrevlex")
f1 = x
f2 = x^2 - y
I = ideal(f1,f2)
G = I.groebner_basis()
G

[x, y]

In [15]:
mac_matrix([f1,f2],2,R)

[x^2, x*y, y^2, x, y, 1]


[ 0  0  0  1  0  0]
[ 1  0  0  0  0  0]
[ 0  1  0  0  0  0]
[ 1  0  0  0 -1  0]

In [16]:
mac_grob([f1,f2],R)

0 [0] True False
1 [x] True False
2 [x^2, x*y, x, y] True True


(2, [x^2, x*y, x, y])

In [18]:
ideal(x^2, x*y).groebner_basis()

[x^2, x*y]

In [47]:
R.<x,y> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2 - 1
f2 = x*y + x
mac_matrix([f1,f2], 3, R)

[x^3, x^2*y, x*y^2, y^3, x^2, x*y, y^2, x, y, 1]


[ 0  0  0  0  1  0  0  0  0 -1]
[ 0  0  0  0  0  1  0  1  0  0]
[ 1  0  0  0  0  0  0 -1  0  0]
[ 0  1  0  0  0  0  0  0 -1  0]
[ 0  1  0  0  1  0  0  0  0  0]
[ 0  0  1  0  0  1  0  0  0  0]

In [23]:
mac_matrix([f1,f2], 3, R).rref()

[x^3, x^2*y, x*y^2, y^3, x^2, x*y, y^2, x, y, 1]


[ 1  0  0  0  0  0  0 -1  0  0]
[ 0  1  0  0  0  0  0  0  0  1]
[ 0  0  1  0  0  0  0 -1  0  0]
[ 0  0  0  0  1  0  0  0  0 -1]
[ 0  0  0  0  0  1  0  1  0  0]
[ 0  0  0  0  0  0  0  0  1  1]

In [24]:
ideal(f1,f2).groebner_basis()

[x^2 - 1, y + 1]

In [27]:
x in ideal(f1,f2)

False

In [33]:
mac_grob([f1,f2], R)[1][0].lt()

0 [0] True False
1 [0] True False
2 [x^2 - 1, x*y + x] False True
3 [x^3 - x, x^2*y + 1, x*y^2 - x, x^2 - 1, x*y + x, y + 1] True True


x^3

In [29]:
mac_grob([1],R)

0 [1] True True


(0, [1])

In [48]:
# used to test assertion that in(d) stabilizes

def mac_grob_bad(f,ring):
    for test_deg in range(20):
        B = mac_basis(f, test_deg, ring)
        lt_ideal = ideal([p.lt() for p in B])
        grob_test = lt_ideal.groebner_basis()
        print(test_deg, B, grob_test)

In [49]:
mac_grob_bad([f1,f2],R)

0 [0] [0]
1 [0] [0]
2 [x^2 - 1, x*y + x] [x^2, x*y]
3 [x^3 - x, x^2*y + 1, x*y^2 - x, x^2 - 1, x*y + x, y + 1] [x^2, y]
4 [x^4 - 1, x^3*y + x, x^2*y^2 - 1, x*y^3 + x, x^3 - x, x^2*y + 1, x*y^2 - x, x^2 - 1, x*y + x, y^2 - 1, y + 1] [x^2, y]
5 [x^5 - x, x^4*y + 1, x^3*y^2 - x, x^2*y^3 + 1, x*y^4 - x, x^4 - 1, x^3*y + x, x^2*y^2 - 1, x*y^3 + x, x^3 - x, x^2*y + 1, x*y^2 - x, y^3 + 1, x^2 - 1, x*y + x, y^2 - 1, y + 1] [x^2, y]
6 Polynomial Sequence with 24 Polynomials in 2 Variables [x^2, y]
7 Polynomial Sequence with 32 Polynomials in 2 Variables [x^2, y]
8 Polynomial Sequence with 41 Polynomials in 2 Variables [x^2, y]
9 Polynomial Sequence with 51 Polynomials in 2 Variables [x^2, y]
10 Polynomial Sequence with 62 Polynomials in 2 Variables [x^2, y]
11 Polynomial Sequence with 74 Polynomials in 2 Variables [x^2, y]
12 Polynomial Sequence with 87 Polynomials in 2 Variables [x^2, y]
13 Polynomial Sequence with 101 Polynomials in 2 Variables [x^2, y]
14 Polynomial Sequence with 116 Polynom

In [50]:
S.<x1,x2,x3> = PolynomialRing(GF(5), order = "lex")
f1 = x3^2 - x2
f2 = x2^3 - x1
mac_grob_bad([f1, f2, x1^5 - x1, x2^5 - x2, x3^5 - x3], S) # further numerical evidence

0 [0] [0]
1 [0] [0]
2 [x2 - x3^2] [x2]
3 [x1*x2 - x1*x3^2, x1 - x2^3, x2^2 - x2*x3^2, x2*x3 - x3^3, x2 - x3^2] [x1, x2]
4 [x1^2*x2 - x1^2*x3^2, x1^2 - x1*x2^3, x1*x2^2 - x1*x2*x3^2, x1*x2*x3 - x1*x3^3, x1*x2 - x2^4, x1*x3^2 - x2^4, x1*x3 - x2^3*x3, x1 - x2^2*x3^2, x2^3 - x2^2*x3^2, x2^2*x3 - x2*x3^3, x2^2 - x3^4, x2*x3^2 - x3^4, x2*x3 - x3^3, x2 - x3^2] [x1, x2]
5 Polynomial Sequence with 32 Polynomials in 3 Variables [x1, x2, x3^5]
6 Polynomial Sequence with 62 Polynomials in 3 Variables [x1, x2, x3^5]
7 Polynomial Sequence with 104 Polynomials in 3 Variables [x1, x2, x3^5]
8 Polynomial Sequence with 155 Polynomials in 3 Variables [x1, x2, x3^5]
9 Polynomial Sequence with 214 Polynomials in 3 Variables [x1, x2, x3^5]
10 Polynomial Sequence with 281 Polynomials in 3 Variables [x1, x2, x3^5]
11 Polynomial Sequence with 359 Polynomials in 3 Variables [x1, x2, x3^5]
12 Polynomial Sequence with 450 Polynomials in 3 Variables [x1, x2, x3^5]
13 Polynomial Sequence with 555 Polynomials in 3 V

In [94]:
R.<x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2
f2 = x*y
f3 = x*z
f4 = y^3
I = ideal(f1,f2,f3,f4)
res = I._singular_().mres(0)
res.betti()

     1     0     0     0
     0     3     3     1
     0     1     1     0

In [95]:
res

[1]:
   _[1]=x*z
   _[2]=x*y
   _[3]=x^2
   _[4]=y^3
[2]:
   _[1]=y*gen(1)-z*gen(2)
   _[2]=x*gen(1)-z*gen(3)
   _[3]=x*gen(2)-y*gen(3)
   _[4]=y^2*gen(2)-x*gen(4)
[3]:
   _[1]=x*gen(1)-y*gen(2)+z*gen(3)

In [1]:
R.<x,y> = PolynomialRing(GF(101), order = "degrevlex")
f1 = x^2
f2 = x*y
I = ideal(f1,f2)
res = I._singular_().mres(0)
res.betti()

     1     0     0
     0     2     1

In [15]:
x = sqrt((sqrt(2) + 1) / 2) + i*sqrt((sqrt(2) - 1) / 2)
(x^2).expand().simplify()

2*I*sqrt(1/2*sqrt(2) + 1/2)*sqrt(1/2*sqrt(2) - 1/2) + 1

In [15]:
R.<x,y,z> = PolynomialRing(QQ, order = "degrevlex")
f1 = x^2 + y^2 
f2 = x*y^2
f3 = y*z
I = ideal(f1,f2,f3)
I._singular_().mres(0).betti()

     1     0     0     0
     0     2     0     0
     0     1     2     0
     0     0     1     1